In [1]:
import pandas as pd

In [62]:
pelis = pd.read_csv('pelis_combinado.csv')

In [3]:
pelis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 619 entries, 0 to 618
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Nombre         619 non-null    object
 1   Fecha          619 non-null    object
 2   Tipo           619 non-null    object
 3   Horario        619 non-null    object
 4   FormatoImagen  619 non-null    object
 5   FormatoIdioma  619 non-null    object
 6   Cine           619 non-null    object
dtypes: object(7)
memory usage: 34.0+ KB


In [63]:
pelis['Fecha'] = pd.to_datetime(pelis['Fecha'])
pelis.drop(columns=['Tipo'], inplace=True)

In [64]:
pelis.sort_values(by='Fecha', inplace=True)

pelis.set_index('Fecha', inplace=True)

In [6]:
pelis

,Nombre,Horario,FormatoImagen,FormatoIdioma,Cine
Fecha,,,,,
2025-09-14,EL CUADRO ROBADO,15:10,2D,Subtitulado,Cines del Centro
2025-09-14,El conjuro 4: últimos ritos,22:00,2D,Doblada,Showcase
2025-09-14,HOMO ARGENTUM,15:00,2D,Español,Cines del Centro
2025-09-14,El conjuro 4: últimos ritos,21:20,2D,Doblada,Showcase
2025-09-14,El conjuro 4: últimos ritos,19:45,2D,Doblada,Showcase
...,...,...,...,...,...
2025-10-27,PARANORMAN 15 ANIVERSARIO,18:00,2D,Español,Cinemark Hoyts
2025-10-28,PARANORMAN 15 ANIVERSARIO,15:35,3D,Español,Cinemark Hoyts
2025-10-28,PARANORMAN 15 ANIVERSARIO,18:00,2D,Español,Cinemark Hoyts


---

In [7]:
print(pelis['Nombre'].unique())

['EL CUADRO ROBADO' 'El conjuro 4: últimos ritos' 'HOMO ARGENTUM'
 'Otro viernes de locos' 'Homo argentum' 'La mujer de la fila'
 'La hora de la desaparición'
 'Demon Slayer: Kimetsu no Yaiba Castillo Infinito'
 'Demon Slayer: Castillo Infinito' 'El Conjuro: Los Ultimos Ritos'
 'Together: juntos hasta la muerte' 'El Conjuro: Los Ultimos Ritos 2D'
 'Together: Juntos hasta la muerte' 'Verano Trippin' 'Homo Argentum'
 'TOY STORY' 'DEMON SLAYER:\r\nKIMETSU NO YAIBA\r\nCASTILLO INFINITO'
 'EL CONJURO 4: ÚLTIMO RITO' 'TOGETHER : JUNTOS HASTA LA MUERTE'
 'LA MUJER DE LA FILA' 'EL CONJURO 4 : ULTIMOS RITOS'
 'DEMON SLAYER CASTILLO INFINITO' 'TOGETHER: JUNTOS\r\nHASTA LA MUERTE'
 'Los 4 Fantásticos: Primeros pasos' 'Los tipos malos 2'
 'Mascotas al rescate' 'LOS 4 FANTASTICOS' 'LOS TIPOS MALOS 2'
 'MASCOTAS AL RESCATE' '200% Lobo' 'Toy Story (RE)'
 'Demon Slayer Kimetsu no Yaiba Castillo Infinito' 'Toy Story'
 'TOY STORY 1995 ( RE ESTRENO )' 'Los Cuatro Fantásticos: primeros pasos'
 '200% lobo'

In [22]:
# ACTUALMENTE NO ESTOY USANDO ESTO

import re
from unidecode import unidecode
from fuzzywuzzy import fuzz

def normalize(text):
    """
    Normalize the input string by converting to lowercase, 
    removing extra whitespaces, and stripping punctuation.
    """

    # Lista de palabras "ruido" a eliminar
    STOPWORDS = [
        "2d", "3d", "imax", "aniversario", "remastered", "re estreno", "tour",
        "re-estreno", "live", "estreno", "version", "extendida", "remasterizada", "the"
    ]
    
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = re.sub(r"\([^)]*\)", "", text) # Eliminar todo lo que esté entre paréntesis
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = unidecode(text)    # Remove accents
    for w in STOPWORDS:
        text = re.sub(rf"\b{w}\b", "", text) # Remove stopwords
    return text.strip()

def unify_movie_names(df, column_name='Nombre', similarity_threshold=55, new_col_name='Nombre_normalizado'):
    """
    Unify movie names in a DataFrame by creating a mapping of movie titles that are similar.
    
    Parameters:
        df (pd.DataFrame): The input DataFrame containing movie names.
        column_name (str): The name of the column with the movie names to be normalized.
        similarity_threshold (int): The similarity threshold (0-100) used by fuzzywuzzy's ratio to classify
                                    two titles as equivalent. Default is 80.
        new_col_name (str): The name of the new column that will store the unified movie names.
    
    Returns:
        pd.DataFrame: The DataFrame with an additional column of unified movie names.
        dict: A mapping dictionary where each original movie title is mapped to its canonical title.
    """
    mapping = {}
    canonical_titles = []
    
    # Retrieve the unique titles in the given column
    unique_titles = df[column_name].unique()
    
    # Process each unique title
    for title in unique_titles:
        norm_title = normalize(title)
        matched = False
        # Compare with already-established canonical titles
        for canonical in canonical_titles:
            if fuzz.token_set_ratio(norm_title, normalize(canonical)) >= similarity_threshold: #len(set(norm_title.split()) & set(normalize(canonical).split())) >= 2: #
                mapping[title] = canonical
                matched = True
                break
        if not matched:
            # If no similar title is found, this title becomes a new canonical title
            canonical_titles.append(title)
            mapping[title] = title
    
    # Map the canonical titles to the new column in the DataFrame
    df[new_col_name] = df[column_name].map(mapping)
    
    return df, mapping

    
   

In [65]:
import re
import unicodedata
from random import shuffle
# ---- opcional: pip install spacy nltk
from fuzzywuzzy import fuzz

# Intentamos usar spaCy (mejor lematización en español). Si no está, usamos un fallback.
try:
    import spacy
    nlp = spacy.load("es_core_news_sm")
except Exception:
    nlp = None
    try:
        from nltk.stem.snowball import SnowballStemmer
        stemmer = SnowballStemmer("spanish")
    except Exception:
        stemmer = None

# stopwords para comparación (NO necesariamente para el título canónico)
MATCH_STOPWORDS = {
    "el","la","los","las","the","y","de","del","en","con","por","para","al"
}

NUM_WORDS = {
    "cero":"0","uno":"1","una":"1","dos":"2","tres":"3","cuatro":"4","cinco":"5",
    "seis":"6","siete":"7","ocho":"8","nueve":"9","diez":"10"
}

# Stopwords extra (no aparecen en el título canónico)
EXTRA_STOPWORDS = {
    "remastered", "imax", "3d", "2d", "aniversario", "extendida",
    "estreno", "re", "re-estreno", "live", "version", "tour", "remasterizada",
    "el","la","los","las","the","y","de","del","en","con","por","para","al" # <--- ojo con esto: los estoy eliminando
}

def remove_accents(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def normalize_text(s):
    s = s.lower()
    s = remove_accents(s)
    # quitar todo lo que esté entre paréntesis
    s = re.sub(r"\([^)]*\)", " ", s)
    # reemplazar puntuación por espacio (mejor que eliminar para no pegar palabras)
    s = re.sub(r"[^\w\s]", " ", s)
    # normalizar espacios
    s = re.sub(r"\s+", " ", s).strip()
    # quitar stopwords extra
    toks = s.split()
    toks = [t for t in toks if t not in EXTRA_STOPWORDS]
    s = " ".join(toks)
    # convertir palabras numerales simples (cuatro -> 4)
    for w, d in NUM_WORDS.items():
        s = re.sub(rf"\b{w}\b", d, s)
    return s

def lemmatize_token(tok):
    if nlp:
        doc = nlp(tok)
        return doc[0].lemma_
    elif stemmer:
        return stemmer.stem(tok)
    else:
        # fallback muy simple: si termina en 's' y es largo, quitar plural final
        if len(tok) > 3 and tok.endswith("s"):
            return tok[:-1]
        return tok

def tokens_and_lemmas(title):
    norm = normalize_text(title)
    toks = re.findall(r"\w+", norm)
    lemmas = [lemmatize_token(t) for t in toks]
    return toks, lemmas

def numbers_in_tokens(toks):
    return {t for t in toks if t.isdigit()}

# regla de similitud
def titles_similar(a, b, token_ratio_thresh=85, jaccard_thresh=0.35):
    toks_a, lem_a = tokens_and_lemmas(a)
    toks_b, lem_b = tokens_and_lemmas(b)

    # bloquear si hay números (secuela/año) y difieren
    nums_a = numbers_in_tokens(toks_a)
    nums_b = numbers_in_tokens(toks_b)
    if nums_a and nums_b and nums_a != nums_b:
        return False

    # core tokens (quitamos stopwords para comparar "núcleos")
    core_a = {t for t in lem_a if t not in MATCH_STOPWORDS and len(t) >= 3}
    core_b = {t for t in lem_b if t not in MATCH_STOPWORDS and len(t) >= 3}

    # Jaccard sobre lemmas (robusto contra plurales si se lematiza)
    inter = core_a & core_b
    union = core_a | core_b
    jaccard = (len(inter) / len(union)) if union else 0.0

    # score fuzzy token-based (ignora orden y repeticiones)
    score = fuzz.token_set_ratio(" ".join(lem_a), " ".join(lem_b))

    # regla combinada:
    # - si fuzzy alto y hay al menos UN token "largo" en común -> match
    # - o fuzzy muy alto (>=92) aun si no hay tokens comunes (casos raros)
    long_token_overlap = any(len(t) >= 4 for t in inter)
    if score >= token_ratio_thresh and (long_token_overlap or jaccard >= jaccard_thresh):
        return True
    if score >= 92:
        return True
    return False


def unify_movie_names(df, column_name='Nombre', similarity_threshold=55, new_col_name='Nombre_normalizado'):
    mapping = {}
    canonical_titles = []
    
    # Retrieve the unique titles in the given column
    unique_titles = df[column_name].unique()
    shuffle(unique_titles)
    
    # Process each unique title
    for title in unique_titles:
        matched = False
        # Compare with already-established canonical titles
        for canonical in canonical_titles:
            if titles_similar(title, canonical):
                mapping[title] = canonical
                matched = True
                break
        if not matched:
            # If no similar title is found, this title becomes a new canonical title
            canonical_titles.append(title)
            mapping[title] = title
    
    # Map the canonical titles to the new column in the DataFrame
    df[new_col_name] = df[column_name].map(mapping)
    
    return df, mapping

In [68]:
# Apply the function
minimo_pelis_unicas = 10**3
mejor_mapping = None

for i in range(30):
    pelis, mapping = unify_movie_names(pelis)
    pelis_unicas = len(pelis['Nombre_normalizado'].unique())
    if pelis_unicas < minimo_pelis_unicas:
        minimo_pelis_unicas = pelis_unicas
        mejor_mapping = mapping.copy()


In [69]:
sorted_mapping = dict(sorted(mapping.items()))
sorted_mapping

{'200 % LOBO': '200 % LOBO',
 '200% Lobo': '200 % LOBO',
 '200% lobo': '200 % LOBO',
 'BTS 2016 LIVE THE MOST BEAUTIFUL MOMENT': 'BTS 2016 Live The Most Beautiful Moment in Life On Stage: Epilogue Remastered',
 'BTS 2016 Live The Most Beautiful Moment in Life On Stage: Epilogue Remastered': 'BTS 2016 Live The Most Beautiful Moment in Life On Stage: Epilogue Remastered',
 'BTS 2016 Live The Most Beautiful Moment: Epilogue': 'BTS 2016 Live The Most Beautiful Moment in Life On Stage: Epilogue Remastered',
 'BTS 2017 LIVE TRILOGY EPISODE III': 'BTS 2017 Live Trilogy EPISODE III THE WINGS TOUR THE FINAL Remastered',
 'BTS 2017 Live Trilogy EPISODE III THE WINGS TOUR THE FINAL Remastered': 'BTS 2017 Live Trilogy EPISODE III THE WINGS TOUR THE FINAL Remastered',
 'BTS 2017 THE WINGS TOUR THE FINAL Remastered': 'BTS 2017 Live Trilogy EPISODE III THE WINGS TOUR THE FINAL Remastered',
 'BTS 2019 WORLD TOUR LOVE YOURSELF': 'BTS 2019 WORLD TOUR LOVE YOURSELF',
 'BTS 2019 WORLD TOUR ‘LOVE YOURSELF:

In [11]:
pelis.head(50)

,Nombre,Horario,FormatoImagen,FormatoIdioma,Cine,Nombre_normalizado
Fecha,,,,,,
2025-09-14,EL CUADRO ROBADO,15:10,2D,Subtitulado,Cines del Centro,EL CUADRO ROBADO
2025-09-14,El conjuro 4: últimos ritos,22:00,2D,Doblada,Showcase,El conjuro 4: últimos ritos
2025-09-14,HOMO ARGENTUM,15:00,2D,Español,Cines del Centro,HOMO ARGENTUM
2025-09-14,El conjuro 4: últimos ritos,21:20,2D,Doblada,Showcase,El conjuro 4: últimos ritos
2025-09-14,El conjuro 4: últimos ritos,19:45,2D,Doblada,Showcase,El conjuro 4: últimos ritos
2025-09-14,El conjuro 4: últimos ritos,22:20,2D,Subtitulado,Showcase,El conjuro 4: últimos ritos
2025-09-14,HOMO ARGENTUM,17:30,2D,Español,Cines del Centro,HOMO ARGENTUM
2025-09-14,El conjuro 4: últimos ritos,21:40,2D,Subtitulado,Showcase,El conjuro 4: últimos ritos
2025-09-14,El conjuro 4: últimos ritos,19:25,2D,Subtitulado,Showcase,El conjuro 4: últimos ritos


In [71]:
def unificar_titulos(text):
    text = text.upper()
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

pelis['Nombre_normalizado'] = pelis['Nombre_normalizado'].apply(unificar_titulos)

In [72]:
pelis.drop(columns=['Nombre'], inplace=True)
pelis.rename(columns={'Nombre_normalizado': 'Nombre'}, inplace=True)

---

In [73]:
def convert_caps_to_title(text):
    """
    Convert any word in all CAPS to a version with only the first letter capitalized.
    
    Parameters:
        text (str): The input string to process.
        
    Returns:
        str: The processed string with all-CAPS words converted.
    """
    def replacer(match):
        word = match.group(0)
        # Check if the entire word is uppercase (this works with Unicode letters as well)
        if word.isupper():
            return word.capitalize()
        return word  # Return unchanged if not fully uppercase
    
    # This regex will match any word composed of word characters.
    # The re.UNICODE flag ensures Unicode characters are considered word characters.
    return re.sub(r'\b\w+\b', replacer, text, flags=re.UNICODE)


def normalize_formato_idioma(df, column_name='FormatoIdioma'):
    """
    Normalizes the words in CAPs within the given column of the DataFrame so that 
    they use only the first letter in caps.
    
    Parameters:
        df (pd.DataFrame): The DataFrame containing the column to process.
        column_name (str): The name of the column to apply the normalization on.
    
    Returns:
        pd.DataFrame: The DataFrame with the updated column.
    """
    # Ensure we work with string data by converting the column values
    df[column_name] = df[column_name].astype(str).apply(convert_caps_to_title)
    return df


In [74]:
# Apply the normalization function on the 'FormatoIdioma' column
pelis = normalize_formato_idioma(pelis, column_name='FormatoIdioma')

In [75]:
pelis.loc[pelis['FormatoIdioma'] != 'Subtitulado', 'FormatoIdioma'] = 'Castellano'

In [76]:
pelis.head(50)

,Horario,FormatoImagen,FormatoIdioma,Cine,Nombre
Fecha,,,,,
2025-09-14,15:10,2D,Subtitulado,Cines del Centro,EL CUADRO ROBADO
2025-09-14,19:40,2D,Castellano,Cines del Centro,HOMO ARGENTUM
2025-09-14,22:00,2D,Castellano,Monumental,TOGETHER: JUNTOS HASTA LA MUERTE
2025-09-14,21:00,2D,Castellano,Monumental,LA MUJER DE LA FILA
2025-09-14,22:05,2D,Castellano,Monumental,EL CONJURO 4: ÚLTIMOS RITOS
2025-09-14,20:40,2D,Castellano,Monumental,EL CONJURO 4: ÚLTIMOS RITOS
2025-09-14,20:30,2D,Subtitulado,Monumental,DEMON SLAYER: KIMETSU NO YAIBA CASTILLO INFINITO
2025-09-14,21:45,2D,Castellano,Monumental,HOMO ARGENTUM
2025-09-14,19:40,2D,Castellano,Monumental,HOMO ARGENTUM


---

# fin

In [77]:
print(pelis['Nombre'].unique())

['EL CUADRO ROBADO' 'HOMO ARGENTUM' 'TOGETHER: JUNTOS HASTA LA MUERTE'
 'LA MUJER DE LA FILA' 'EL CONJURO 4: ÚLTIMOS RITOS'
 'DEMON SLAYER: KIMETSU NO YAIBA CASTILLO INFINITO'
 'OTRO VIERNES DE LOCOS' 'LA HORA DE LA DESAPARICIÓN'
 'TOY STORY 30 ANIVERSARIO' 'MASCOTAS AL RESCATE' '200 % LOBO'
 'LOS CUATRO FANTÁSTICOS: PRIMEROS PASOS' 'LOS TIPOS MALOS 2'
 'VERANO TRIPPIN' 'TOY STORY 1995 ( RE ESTRENO )'
 'DAVID GILMOUR LIVE AT THE CIRCUS MAXIMUS, ROME'
 'UNA VELADA CON DUA LIPA' 'BELÉN' 'PAPÁ X DOS' 'F1'
 'EL GRAN VIAJE DE TU VIDA' 'LA NOVICIA REBELDE 60° ANIVERSARIO'
 'BTS 2016 LIVE THE MOST BEAUTIFUL MOMENT IN LIFE ON STAGE: EPILOGUE REMASTERED'
 'BTS 2017 LIVE TRILOGY EPISODE III THE WINGS TOUR THE FINAL REMASTERED'
 'BTS 2019 WORLD TOUR LOVE YOURSELF' 'BTS 2021 MUSTER SOWOOZOO REMASTERED'
 'MITSKI: THE LAND' 'PARANORMAN 15 ANIVERSARIO']


In [78]:
pelis.to_csv('pelis_combinado_clean.csv', index=True)